In [20]:
from hydra import compose, initialize
from omegaconf import OmegaConf

initialize(config_path='conf/')

/tmp/ipykernel_3161906/2329082962.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path='conf/')


ValueError: GlobalHydra is already initialized, call GlobalHydra.instance().clear() if you want to re-initialize

In [21]:
cfg = compose(config_name='config')

import tensorflow as tf
import math
import pandas as pd
import numpy as np

from pathlib import Path

from src import datasets 

In [22]:
# Load the data
uni_cfg = cfg.dataset
dat_cfg = cfg.dataset.args.test
labels = pd.read_csv(cfg.dataset.labels)
labels_len = len(labels)
batch_size = cfg.mode.batch_size

del labels

dataset = datasets.WayneCroppedDataset(
    data_idx = range(labels_len),
    shuffle=dat_cfg.shuffle,
    balance=dat_cfg.balance,
    batch_size=batch_size,
    data_dir=uni_cfg.data_dir,
    labels = uni_cfg.labels,
    channels = uni_cfg.channels
)

assert int(math.ceil(labels_len / batch_size)) == len(dataset)

In [23]:
# Load model
model_path = '/work/users/o/e/oem/PHASER/saved_models/alexnet_defaults/2024-07-23/epoch_6_vloss_0.15.h5'
model_path = Path(model_path)

model = tf.keras.models.load_model(model_path)

In [35]:
colnames = ['cell_id', 'prediction', 'true_lab']
phase_labels = ['G1', 'S', 'G2']
results_df = pd.DataFrame(columns=colnames)

for s, step in enumerate(dataset):
    
    if len(step[0]) == 0:
        break
    
    predictions = model.predict(step[0])
    predictions = np.argmax(predictions, axis=-1)
    predictions = np.array([phase_labels[i] for i in predictions])
    
    true_lab = step[1].numpy()
    true_lab = np.argmax(true_lab, axis=-1)
    true_lab = np.array([phase_labels[i] for i in true_lab])
    
    cell_ids = step[-1].numpy()
    
    stacked = np.vstack([cell_ids, predictions, true_lab])
    
    temp_df = pd.DataFrame(stacked.T, columns=colnames)
    
    results_df = pd.concat([results_df, temp_df], ignore_index=True)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

In [38]:
results_df['cell_id'] = results_df['cell_id'].str.decode('utf-8')

results_df

,cell_id,prediction,true_lab
0,cell_7449,G2,G2
1,cell_7327,G1,G1
2,cell_2095,G1,G1
3,cell_1318,G1,G1
4,cell_3606,G1,G1
...,...,...,...
5870,cell_5027,G1,G1
5871,cell_8540,G1,G1
5872,cell_5751,G2,G2
5873,cell_7434,G1,G1


In [39]:
# Save the results
save_results = '/work/users/o/e/oem/PHASER/results/inference_on_all_cells.csv'
save_results = Path(save_results)
save_results.parent.mkdir(exist_ok=True, parents=True)
results_df.to_csv(save_results)